# 📘 Algoritmo Deep Q-Network (DQN)

## 🧠 Introducción

El algoritmo **Deep Q-Network (DQN)** fue propuesto por DeepMind en 2013-2015 y marcó un hito al lograr jugar videojuegos de Atari a nivel humano. Combina el **Q-Learning** tradicional con redes neuronales profundas para aproximar la función de valor, lo que permite manejar espacios de estados continuos y de alta dimensión.

**Publicación clave:**  
- *Playing Atari with Deep Reinforcement Learning* (Mnih et al., 2013)  
  [ArXiv paper](https://arxiv.org/abs/1312.5602)

![Red](https://miro.medium.com/v2/resize:fit:1153/1*emv9eFMbGODD4gnITjfwcQ.png)

![Red2](https://www.researchgate.net/publication/358148766/figure/fig1/AS:1116926097014784@1643307341035/DQN-training-schematic-showing-policy-network-target-network-and-experience-replay.png)
---

## ⚙️ Características principales de DQN

- Utiliza una **red neuronal** para aproximar la función Q:  
  $
  Q(s, a; \theta) \approx Q^*(s, a)
  $
- Emplea un **replay buffer** para romper la correlación temporal entre experiencias.
- Usa una **red objetivo (target network)** para estabilizar el entrenamiento.
- Algoritmo **off-policy**, basado en Q-Learning.
- Estrategia de exploración: **ε-greedy**.

---

## 🧮 Ecuaciones clave

### 1. **Actualización de Q-Learning**

La función de valor Q se actualiza usando la ecuación de Bellman:

$
Q_{\text{target}} = r + \gamma \max_{a'} Q(s', a'; \theta^-)
$

$
\mathcal{L}(\theta) = \mathbb{E}_{(s,a,r,s') \sim \mathcal{D}} \left[ \left(Q(s,a;\theta) - Q_{\text{target}}\right)^2 \right]
$

Donde:
- $ \theta $: parámetros de la red principal.
- $ \theta^- $: parámetros de la red objetivo (actualizados cada cierto tiempo).
- $ \gamma $: factor de descuento.
- $ \mathcal{D} $: buffer de experiencia.

---

## 🖼️ Arquitectura (representación visual)

Una arquitectura típica de DQN para Atari incluye:

- Preprocesamiento de imágenes.
- CNN para extraer características del estado.
- Capas densas para generar valores Q.

**Imagen representativa de la arquitectura:**

![DQN Architecture](https://www.researchgate.net/publication/318184943/figure/fig1/AS:812127505879042@1570637694827/DQN-architecture-for-end-to-end-learning-of-Atari-2600-game-plays.png)  
Crédito: [Denny Britz - RL GitHub](https://github.com/dennybritz/reinforcement-learning)

---

## 📦 Experience Replay Buffer

**¿Qué es?**  
Una estructura de datos (FIFO o circular) que almacena transiciones \( (s, a, r, s') \) observadas durante las interacciones del agente con el entorno.

### Ventajas:
- Rompe la correlación entre experiencias consecutivas.
- Permite reutilizar datos para mejorar la eficiencia.
- Mejora la estabilidad del entrenamiento.

### Implementación básica:
- Tamaño fijo.
- Muestreo aleatorio mini-batch.
- Reemplazo cuando se llena.

---

## 📜 Pseudocódigo: Algoritmo de entrenamiento de $Q$-Learning

Usaremos el siguiente algoritmo para entrenar la red.

* Inicializar la memoria $D$
* Inicializar la red de valores de acción $Q$ con pesos aleatorios
* **Para** episodio $\leftarrow 1$ **hasta** $M$ **hacer**
  * Observar $s_0$
  * **Para** $t \leftarrow 0$ **hasta** $T-1$ **hacer**
     * MUESTREO
     * Con probabilidad $\epsilon$ seleccionar una acción aleatoria $a_t$, de lo contrario seleccionar $a_t = \mathrm{argmax}_a Q(s_t,a)$
     * Ejecutar acción $a_t$ en el simulador y observar la recompensa $r_{t+1}$ y el nuevo estado $s_{t+1}$
     * Almacenar la transición $<s_t, a_t, r_{t+1}, s_{t+1}>$ en la memoria $D$
     * ENTRENAMIENTO
     * Muestrear un mini-lote aleatorio de $D$: $<s_j, a_j, r_j, s'_j>$
     * Establecer $Q^t_j = r_j$ si el episodio termina en $j+1$, de lo contrario establecer $Q^t_j = r_j + \gamma \max_{a'}{Q(s'_j, a')}$
     * Realizar un paso de descenso por gradiente con la pérdida $(\hat{Q}_j - Q(s_j, a_j))^2$
  * **fin para**
* **fin para**

¡Se recomienda (y se fomenta!) tomar el tiempo para extender este código e implementar algunas de las mejoras que discutimos en la lección, incluyendo objetivos fijos de $Q$, Double DQNs, reproducción priorizada y/o redes dueling.


In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    COLAB = True
    print("Nota: Usando Google CoLab")
except:
    print("Nota: Usando JupyterNotebook")
    COLAB = False

Mounted at /content/drive/
Nota: Usando Google CoLab


In [3]:
if COLAB:
  !apt update && apt install xvfb
  !pip install gym-notebook-wrapper
  !pip install flappy-bird-gymnasium
  !pip install renderlab
  !pip install opencv-python
  !pip install torch

from IPython.display import clear_output
import warnings

warnings.filterwarnings('ignore')
clear_output()

## Entorno sin aprendizaje

In [4]:
import flappy_bird_gymnasium
import gymnasium
import renderlab as rl

# Documentacion https://github.com/markub3327/flappy-bird-gymnasium

env = gymnasium.make("FlappyBird-v0", render_mode="rgb_array", use_lidar=False)
env = rl.RenderFrame(env, "./output") # Inicializa la carpeta para render

obs, _ = env.reset()
while True:
    # Next action:
    # (feed the observation to your agent here)
    action = env.action_space.sample()

    # Processing:
    obs, reward, terminated, _, info = env.step(action)

    # Checking if the player is still alive
    if terminated:
        break
env.play()
env.close()

Moviepy - Building video temp-{start}.mp4.
Moviepy - Writing video temp-{start}.mp4



Moviepy - Done !
Moviepy - video ready temp-{start}.mp4


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np

class DQN(nn.Module):
    def __init__(self, input_size, output_size, fc1_nodes=512):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_size, fc1_nodes)
        self.fc2 = nn.Linear(fc1_nodes, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def train_dqn(env_id, config):
  # Inicialización del entorno
  env = gymnasium.make(env_id, **config['env_make_params'])

  # Inicialización del agente DQN
  input_size = env.observation_space.shape[0]
  output_size = env.action_space.n
  policy_net = DQN(input_size, output_size, config['fc1_nodes'])
  target_net = DQN(input_size, output_size, config['fc1_nodes'])
  target_net.load_state_dict(policy_net.state_dict())
  optimizer = optim.Adam(policy_net.parameters(), lr=config['learning_rate_a'])

  # Replay memory
  replay_memory = []

  # Hiperparámetros
  epsilon = config['epsilon_init']

  # Bucle principal de entrenamiento
  for episode in range(1000): # Número de episodios
    obs, _ = env.reset()
    state = torch.tensor(obs, dtype=torch.float32)
    total_reward = 0

    for t in range(1000): # Número de pasos por episodio
      if random.random() < epsilon:
        action = env.action_space.sample()
      else:
        with torch.no_grad():
          q_values = policy_net(state)
          action = torch.argmax(q_values).item()

      next_obs, reward, terminated, truncated, info = env.step(action)
      next_state = torch.tensor(next_obs, dtype=torch.float32)
      done = terminated or truncated
      replay_memory.append((state, action, reward, next_state, done))

      state = next_state
      total_reward += reward
      if done:
        break

    # Actualización de la red
    if len(replay_memory) >= config['mini_batch_size']:
        minibatch = random.sample(replay_memory, config['mini_batch_size'])
        # Aquí debes implementar la actualización de los pesos del DQN
        # basándote en el minibatch.

        # Obtener un mini-batch aleatorio de la memoria de repeticiones
        minibatch = random.sample(replay_memory, config['mini_batch_size'])

        # Listas para estados, acciones, recompensas y estados siguientes
        states, actions, rewards, next_states, dones = zip(*minibatch)

        # Convertir a tensores (en caso de usar PyTorch o TensorFlow)
        states = torch.tensor(states, dtype=torch.float32)
        actions = torch.tensor(actions, dtype=torch.int64).unsqueeze(1)  # Dimensión adicional para indexar
        rewards = torch.tensor(rewards, dtype=torch.float32).unsqueeze(1)
        next_states = torch.tensor(next_states, dtype=torch.float32)
        dones = torch.tensor(dones, dtype=torch.float32).unsqueeze(1)

        # Predecir los valores Q actuales
        current_q_values = policy_net(states).gather(1, actions)  # Selecciona Q(s, a) para las acciones tomadas

        # Calcular los valores Q objetivos
        with torch.no_grad():
            max_next_q_values = target_net(next_states).max(1, keepdim=True)[0]  # Máximo Q(s', a')
            target_q_values = rewards + (config['gamma'] * max_next_q_values * (1 - dones))  # Q(s, a)

        # Calcular la pérdida
        loss = criterion(current_q_values, target_q_values)

        # Actualizar los pesos del modelo
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Actualización de hiperparámetros
    epsilon = max(config['epsilon_min'], epsilon * config['epsilon_decay'])

    if episode % config['network_sync_rate'] == 0:
      target_net.load_state_dict(policy_net.state_dict())


  env.close()
  print("Entrenamiento terminado")

# Ejemplo de uso con la configuración proporcionada
config = {
    'replay_memory_size': 100000,
    'mini_batch_size': 32,
    'epsilon_init': 1,
    'epsilon_decay': 0.99995,
    'epsilon_min': 0.05,
    'network_sync_rate': 10,
    'learning_rate_a': 0.0001,
    'discount_factor_g': 0.99,
    'stop_on_reward': 100000,
    'fc1_nodes': 512,
    'env_make_params': {
        'use_lidar': False,
    },
}

train_dqn("FlappyBird-v0", config)


ValueError: only one element tensors can be converted to Python scalars

In [ ]:
env = gymnasium.make("FlappyBird-v0", render_mode="rgb_array", use_lidar=True)
env = rl.RenderFrame(env, "./output") # Inicializa la carpeta para render

obs, _ = env.reset()
while True:
    # Next action:
    # (feed the observation to your agent here)
    action = env.action_space.sample()

    # Processing:
    obs, reward, terminated, _, info = env.step(action)

    # Checking if the player is still alive
    if terminated:
        break
env.play()
env.close()